In [14]:
from google.colab import files
uploaded = files.upload()



Saving .env to .env


In [15]:
pip install dotenv

In [16]:
pip install pypdf gradio OpenAI


In [17]:
from dotenv import load_dotenv #for dotenv file
from pypdf import PdfReader #python library for reading pdfs
import requests #for request
from  openai import OpenAI ##open ai client for llms
import os
import gradio as gr ##gradio for easy chat interface

In [18]:
load_dotenv(override=True)  ##for loading dotenv
openai = OpenAI()  ##using open ai gpt otherwise would have to specify more
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"
if(pushover_token):
  print('Pushover token found')
else:
  print('Pushover token unavailable')
if(pushover_url):
  print('Pushover url found')
else:
  print('Pushover url unavailable')
if(pushover_user):
  print('Pushover user found')
else:
  print('Pushover user unavailable')

Pushover token found
Pushover url found
Pushover user found


In [19]:
def push(message):
 '''
 This function is for pushing the message to the pushover :
  it takes the user's user,token,and a url to then post on pushover
 '''
 print(f"Push:{message}")
 payload ={"user":pushover_user,'token':pushover_token, 'message':message}
 requests.post(pushover_url,data=payload)

In [10]:
push('Hey')

Push:Hey


In [20]:
def record_user_details(email,name="name not provided",notes="notes not provided"):
  '''
  This function records the user details and would be called whenever user provides
  his email so that I recieve the message
  '''
  push(f"Recording interest from {name} with email:{email} and notes {notes}")
  return {"recorded":"ok"}

In [21]:
def record_unknown_question(question):
  ''''
  User may definitely ask some questions beyond the reach of knowledge base..
  the agent must then record the question and then send to me on pushover
  so that I could use that as a feedback for training source
  '''
  push(f"Recording question:{question} was asked to me that I couldn't answer")
  return {"recorded unknown":"ok"}

In [22]:
record_user_details_json = {
    "name":"record_user_details",
    "description":"Use this tool to record user details when user is interested in being in touch by giving his email address",
    "parameters":{
        "type":"object",
        "properties":{
            "email":{
              "type":"string",
              "description":"The email address of this user"
            },
          	"name":{
               "type":"string",
               "description":"The name of the user"

            },
            "notes":{
                "type":"string",
                "description":"Addition information of context worth mentioning within the conversation"
            }
        },
        "required":["email"],
        "additionalProperties":False
    }
    }


In [23]:
record_unknown_question_json = {
    "name":"record_unknown_question",
    "description":"Use this tool to record questions that you don't have answers for",
    "parameters":{
        "type":"object",
        "properties":{
            "question":{
                "type":"string",
                "description":"The question that couldn't be answered"

            }
        },
        "required":["question"],
        "additionalProperties":False
    }

}

In [24]:
tools = [{"type":"function","function":record_user_details_json},
         {"type":"function","function":record_unknown_question_json}]

         #These are the tools/functions our ai agent would perform on the basis of results

In [22]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record user details when user is interested in being in touch by giving his email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string', 'description': 'The name of the user'},
     'notes': {'type': 'string',
      'description': 'Addition information of context worth mentioning within the conversation'}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_questions',
   'description': "Use this tool to record questions that you don't have answers for",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['question'],
    'additionalProperties': False}}}]

In [25]:
#The below function takes the list of tools and decides which to run
def handle_tool_calls(tool_calls):
  results=[]
  for tool in tool_calls:
    tool_name =tool.function.name
    arguments = json.loads(tool.function.arguments)
    print(f"Tool called {tool_name}",flush=True)
    if tool_name == 'record_user_details':
      result = record_user_details(**arguments)
    elif tool_name == 'record_unknown_question':
      result = record_unknown_question(**arguments)
    results.append({"role":"tool","content":json.dumps(result),"tool_call_id":tool.id})
  return results



In [26]:
globals()["record_unknown_question"]("this is a really hard question")

Push:Recording question:this is a really hard question was asked to me that I couldn't answer


{'recorded unknown': 'ok'}

In [31]:
from google.colab import files
my_pdf = files.upload()

Saving Profile.pdf to Profile.pdf


In [30]:
from google.colab import files
summary = files.upload()

Saving summary.txt to summary.txt


In [32]:
reader = PdfReader('Profile.pdf')
linkedin = ""
for page in reader.pages:
  text = page.extract_text()
  if(text):
    linkedin += text
with open('summary.txt','r',encoding='utf-8') as f:
  summary = f.read()

name = "Muhammad Mustafa Mubashir"


In [33]:
summary

'I am Mustafa Mubashir , A Computer Science student at UBIT (University of Karachi) in Pakistan. I’m passionate about building things from scratch and love exploring new ways to innovate and grow. 🚀\nCurrently, I work on both mobile and web app development, taking projects from the initial design phase all the way to database design, backend development, and seamless deployment. I’m diving deep into AI/ML, integrating these technologies into my apps, and am even working towards building and deploying AI solutions from the ground up. 🤖\nI follow a holistic approach to learning—mastering one area while gaining a solid understanding of other subfields in Computer Science (think T-shaped, U-shaped—honestly, I don’t remember the exact shape, but you get the idea!). 💡 My main focus is on sharpening my problem-solving, quick learning, and analytical skills, as I believe these are the foundation of everything else. \nEven though I’m working across various complex domains, I still consider myse

In [35]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "
system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [36]:
## The main chat function
def chat(message,history):
  messages =[{"role":"system","content":system_prompt}]+history+[{"role":"user","content":message}]
  done = False
  while not done:
    response = openai.chat.completions.create(model="gpt-4o-mini",messages=messages)
    finish_reason = response.choices[0].finish_reason

    if finish_reason == "tool_calls":
      message = response.choices[0].message
      results = handle_tool_calls(message.tool_calls)
      messages.append(message)
      messages.extent(results)
    else:
      done  = True
  return response.choices[0].message.content

In [37]:
gr.ChatInterface(chat,type='messages').launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1182ed89939025f75e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
